In [ ]:
import numpy as np
from model_age import model_generate_age
from model_gender import model_generate_gender
from model_emotion import model_generate_emotion
from matplotlib.pyplot import imshow
from keras import optimizers
import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
session = tf.Session(config=config)
KTF.set_session(session)

In [ ]:
# Compute the accuracy of the emotion dataset
def compute_accuracy(model, X_test, Y_test):
    Y_test_pred = model.predict(X_test, verbose=1)
    Y_test_pred_arg = np.argmax(Y_test_pred, axis=1)
    count = 0
    for i in range(Y_test.shape[0]):
        if (Y_test[i][Y_test_pred_arg[i]] == np.max(Y_test[i])):
            count += 1
    accuracy_score_test = count / Y_test.shape[0]
    print(accuracy_score_test)

In [ ]:
model_emotion = model_generate_emotion(2)

In [ ]:
model_emotion.load_weights('../logs/weights-emotion.h5',skip_mismatch=True,by_name=True)

In [ ]:
sgd = optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
Adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model_emotion.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
X = np.load('face_X.npy')
Y = np.load('face_Y.npy')
y_ = []
for i in Y:
    y = np.zeros(2,dtype=np.float)
    y[i] = 1.0
    y_.append(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y_, random_state=0, test_size=0.2)

In [ ]:
for i in range(400):
    model_emotion.fit(np.array(x_train),np.array(y_train))
    compute_accuracy(model_emotion, np.array(x_test), np.array(y_test))
model_emotion.save('depressed_model.h5')